In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import urllib
from nltk import sent_tokenize
from nltk import word_tokenize

# Convert

In [2]:
news = pd.read_csv('newzy.csv' , sep = '|')

In [3]:
news.head()

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url
0,39,Google News,"50 Years on, Finding Profit in 'Truth' on JFK ...","Washington Post50 Years on, Finding Profit in ...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
1,38,Google News,"Nearly 10 years later, families and officials ...","Washington PostNearly 10 years later, families...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
2,37,Google News,"Cuccinelli, running hard, has been hobbled in ...","Washington PostCuccinelli, running hard, has b...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
3,36,Google News,Chuck Schumer endorses Hillary Clinton for Pre...,New York Daily NewsChuck Schumer endorses Hill...,11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
4,35,Google News,Sec. Kerry issues apparent warning to Iran - C...,Ahram OnlineSec. Kerry issues apparent warning...,11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...


In [4]:
news.shape

(1026347, 6)

In [10]:
news.doc_source.unique()

array(['Google News', 'PowerLine', 'Reuters', 'Guardian', 'Daily Kos',
       'Drudge Report', 'Real Clear Politics', 'NPR', 'Fox',
       'New York Times', 'Breitbart', 'CNN', 'US News', 'UPI Latest',
       'Politico Magazine', nan], dtype=object)

In [8]:
data = news[news.doc_source.isin(['PowerLine', 'Politico Magazine'])].reset_index(drop = True)

In [11]:
# creat column 'author' from doc_content
data['author'] = data.doc_content.apply(lambda x: str(x).split(')')[0].strip('('))
data.doc_content = data.doc_content.apply(lambda x: str(x).strip(str(x).split(')')[0]).strip(')'))
data.head()

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url,author
0,48,PowerLine,These Zins Are a Dusi,I’ve been falling behind on many regular feat...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...,Steven Hayward
1,49,PowerLine,NY Times Editorial Board Hacked by The Onion,"At least I think it was the Onion. Someone, a...",11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...,John Hinderaker
2,50,PowerLine,The line Obama crossed,Obamacare was enacted on a foundational set o...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...,Scott Johnson
3,51,PowerLine,Sham/Wow,It’s unfair to compare and contrast Barack Ob...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...,Scott Johnson
4,52,PowerLine,A Quarterback Controversy With a Political Twist?,Public Policy Polling ran a survey here in Mi...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...,John Hinderaker


In [2]:
OHCO = ['doc_id', 'sent_num', 'token_num']
#date - y m w, weekly cycles


In [19]:
LIBRARY = data[['doc_source', 'doc_id', 'doc_title', 'author', 'doc_date']].reset_index(drop = True)
#LIBRARY = LIBRARY.reset_index().set_index(OHCO[:2]).drop(columns = 'index')

LIBRARY.head() #doc table??

,doc_source,doc_id,doc_title,author,doc_date
0,PowerLine,48,These Zins Are a Dusi,Steven Hayward,11/03/2013
1,PowerLine,49,NY Times Editorial Board Hacked by The Onion,John Hinderaker,11/03/2013
2,PowerLine,50,The line Obama crossed,Scott Johnson,11/03/2013
3,PowerLine,51,Sham/Wow,Scott Johnson,11/03/2013
4,PowerLine,52,A Quarterback Controversy With a Political Twist?,John Hinderaker,11/03/2013


In [27]:
df = data.doc_content.apply(lambda x: pd.Series(nltk.sent_tokenize(str(x))))
DOC = df.set_index(data.doc_id).stack().to_frame().rename(columns={0:'sent_str'})
DOC

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


sent_str
doc_id                                                       
48      0    I’ve been falling behind on many regular feat...
        1                   So here’s a quick catch up guide.
        2   Old timers of Paso Robles wines all know the D...
        3   The Dusi family’s Zinfandel vineyards go back ...
        4   For decades, if you wanted to find the best Zi...
...                                                       ...
1026333 13  They have repeatedly broken into chants of “Tr...
        14  I took this picture in the East Room last Fall...
        15      At least 95% of those in the room were black.
        16  They loved Trump, and Trump loved them:  Casua...
        17  But alleging that the President of the United ...

[233558 rows x 1 columns]

In [39]:
TOKEN = DOC.sent_str.apply(lambda x: pd.Series(nltk.pos_tag(nltk.word_tokenize(x))))
TOKEN


0                1                 2              3    \
doc_id                                                                      
48      0      (I, PRP)         (’, VBP)          (ve, RB)    (been, VBN)   
        1      (So, RB)       (here, RB)           (’, JJ)       (s, VBP)   
        2    (Old, NNP)    (timers, NNS)          (of, IN)    (Paso, NNP)   
        3     (The, DT)      (Dusi, NNP)      (family, NN)       (’, NNP)   
        4     (For, IN)   (decades, NNS)            (,, ,)       (if, IN)   
...                 ...              ...               ...            ...   
1026333 13  (They, PRP)      (have, VBP)  (repeatedly, RB)  (broken, VBN)   
        14     (I, PRP)      (took, VBD)        (this, DT)  (picture, NN)   
        15     (At, IN)     (least, JJS)          (95, CD)        (%, NN)   
        16  (They, PRP)     (loved, VBD)      (Trump, NNP)         (,, ,)   
        17    (But, CC)  (alleging, VBG)        (that, IN)      (the, DT)   

                         4                 5                 6    \
doc_id                                                             
48      0     (falling, VBG)      (behind, IN)          (on, IN)   
        1            (a, DT)       (quick, JJ)       (catch, NN)   
        2      (Robles, NNP)      (wines, VBZ)         (all, DT)   
        3            (s, NN)  (Zinfandel, NNP)  (vineyards, NNS)   
        4         (you, PRP)     (wanted, VBD)          (to, TO)   
...                      ...               ...               ...   
1026333 13        (into, IN)     (chants, NNS)          (of, IN)   
        14          (in, IN)         (the, DT)       (East, NNP)   
        15          (of, IN)       (those, DT)          (in, IN)   
        16         (and, CC)      (Trump, NNP)      (loved, VBD)   
        17  (President, NNP)          (of, IN)         (the, DT)   

                      7                         8                9    ...  \
doc_id                                                                ...   
48      0      (many, JJ)             (regular, JJ)  (features, NNS)  ...   
        1        (up, RP)               (guide, NN)           (., .)  ...   
        2     (know, VBP)                 (the, DT)      (Dusi, NNP)  ...   
        3       (go, VBP)                (back, RB)     (nearly, RB)  ...   
        4      (find, VB)                 (the, DT)      (best, JJS)  ...   
...                   ...                       ...              ...  ...   
1026333 13       (“, NNP)  (Trump…Trump…Trump, NNP)           (,, ,)  ...   
        14    (Room, NNP)                (last, JJ)      (Fall, NNP)  ...   
        15      (the, DT)                (room, NN)      (were, VBD)  ...   
        16    (them, PRP)                    (:, :)     (Casual, JJ)  ...   
        17  (United, NNP)            (States, NNPS)        (is, VBZ)  ...   

            613  614  615  616  617  618  619  620  621  622  
doc_id                                                        
48      0   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        1   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        2   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        3   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        4   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
1026333 13  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        14  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        15  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        16  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        17  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[233558 rows x 623 columns]

In [40]:
TOKEN = TOKEN.stack().to_frame().rename(columns = {0:'pos_tuple'})
TOKEN

pos_tuple
doc_id                          
48      0  0            (I, PRP)
           1            (’, VBP)
           2            (ve, RB)
           3         (been, VBN)
           4      (falling, VBG)
...                          ...
1026333 17 20  (represents, VBZ)
           21            (a, DT)
           22          (new, JJ)
           23          (low, JJ)
           24             (., .)

[5449229 rows x 1 columns]

In [41]:
TOKEN['pos'] = TOKEN.pos_tuple.apply(lambda x: x[1])
TOKEN['token_str'] = TOKEN.pos_tuple.apply(lambda x: x[0])
TOKEN.index.names = OHCO
TOKEN

pos_tuple  pos   token_str
doc_id  sent_num token_num                                    
48      0        0                   (I, PRP)  PRP           I
                 1                   (’, VBP)  VBP           ’
                 2                   (ve, RB)   RB          ve
                 3                (been, VBN)  VBN        been
                 4             (falling, VBG)  VBG     falling
...                                       ...  ...         ...
1026333 17       20         (represents, VBZ)  VBZ  represents
                 21                   (a, DT)   DT           a
                 22                 (new, JJ)   JJ         new
                 23                 (low, JJ)   JJ         low
                 24                    (., .)    .           .

[5449229 rows x 3 columns]

In [43]:
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')#remove anything that's not words
TOKEN.term_str = TOKEN.term_str.replace('',np.NaN)
TOKEN = TOKEN[~TOKEN.term_str.isna()]
TOKEN #can have duplicate vocab

pos_tuple  pos   token_str    term_str
doc_id  sent_num token_num                                                
48      0        0                   (I, PRP)  PRP           I           i
                 2                   (ve, RB)   RB          ve          ve
                 3                (been, VBN)  VBN        been        been
                 4             (falling, VBG)  VBG     falling     falling
                 5               (behind, IN)   IN      behind      behind
...                                       ...  ...         ...         ...
1026333 17       19             (people, NNS)  NNS      people      people
                 20         (represents, VBZ)  VBZ  represents  represents
                 21                   (a, DT)   DT           a           a
                 22                 (new, JJ)   JJ         new         new
                 23                 (low, JJ)   JJ         low         low

[4669302 rows x 4 columns]

In [44]:
VOCAB = TOKEN.term_str.value_counts().to_frame()\
    .rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'
VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')# num: boolean, if term is number, then set to 1

In [46]:
VOCAB #count each word, and re-organize

,term_str,n,num
term_id,,,
0,0,28,1
1,00,6,1
2,000,4,1
3,000000,1,1
4,00000001,1,1
...,...,...,...
80425,گر,1,0
80426,گروههای,1,0
80427,گزارش,1,0


# Annotate

In [52]:
#STOPWORDS
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1
sw

,dummy
term_str,
i,1
me,1
my,1
myself,1
we,1
...,...
weren't,1
won,1
won't,1


In [53]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [54]:
#add stem
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.term_str.apply(stemmer1.stem)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.term_str.apply(stemmer2.stem)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.term_str.apply(stemmer3.stem)

In [57]:
VOCAB

,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster
term_id,,,,,,,
0,0,28,1,0,0,0,0
1,00,6,1,0,00,00,00
2,000,4,1,0,000,000,000
3,000000,1,1,0,000000,000000,000000
4,00000001,1,1,0,00000001,00000001,00000001
...,...,...,...,...,...,...,...
80425,گر,1,0,0,گر,گر,گر
80426,گروههای,1,0,0,گروههای,گروههای,گروههای
80427,گزارش,1,0,0,گزارش,گزارش,گزارش


In [56]:
TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)
TOKEN

pos_tuple  pos   token_str    term_str  \
doc_id  sent_num token_num                                                   
48      0        0                   (I, PRP)  PRP           I           i   
                 2                   (ve, RB)   RB          ve          ve   
                 3                (been, VBN)  VBN        been        been   
                 4             (falling, VBG)  VBG     falling     falling   
                 5               (behind, IN)   IN      behind      behind   
...                                       ...  ...         ...         ...   
1026333 17       19             (people, NNS)  NNS      people      people   
                 20         (represents, VBZ)  VBZ  represents  represents   
                 21                   (a, DT)   DT           a           a   
                 22                 (new, JJ)   JJ         new         new   
                 23                 (low, JJ)   JJ         low         low   

                            term_id  
doc_id  sent_num token_num           
48      0        0            35204  
                 2            76013  
                 3             9599  
                 4            26831  
                 5             9684  
...                             ...  
1026333 17       19           52831  
                 20           59944  
                 21            3769  
                 22           48179  
                 23           42675  

[4669302 rows x 5 columns]

In [58]:
LIBRARY.to_csv('LIB.csv')
DOC.to_csv('DOC.csv')
TOKEN.to_csv('TOKEN.csv')
VOCAB.to_csv('VOCAB.csv')


# TF-IDF

In [17]:
TOKEN = pd.read_csv('TOKEN.csv')
VOCAB = pd.read_csv('VOCAB.csv')

In [6]:
TOKEN

,doc_id,sent_num,token_num,pos_tuple,pos,token_str,term_str,term_id
0,48,0,0,"('I', 'PRP')",PRP,I,i,35204
1,48,0,2,"('ve', 'RB')",RB,ve,ve,76013
2,48,0,3,"('been', 'VBN')",VBN,been,been,9599
3,48,0,4,"('falling', 'VBG')",VBG,falling,falling,26831
4,48,0,5,"('behind', 'IN')",IN,behind,behind,9684
...,...,...,...,...,...,...,...,...
4669297,1026333,17,19,"('people', 'NNS')",NNS,people,people,52831
4669298,1026333,17,20,"('represents', 'VBZ')",VBZ,represents,represents,59944
4669299,1026333,17,21,"('a', 'DT')",DT,a,a,3769
4669300,1026333,17,22,"('new', 'JJ')",JJ,new,new,48179


In [59]:
VOCAB['pos_max'] = TOKEN.groupby(['term_id', 'pos']).pos.count().unstack().idxmax(1)


In [60]:
VOCAB

,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster,pos_max
term_id,,,,,,,,
0,0,28,1,0,0,0,0,CD
1,00,6,1,0,00,00,00,JJ
2,000,4,1,0,000,000,000,CD
3,000000,1,1,0,000000,000000,000000,NN
4,00000001,1,1,0,00000001,00000001,00000001,CD
...,...,...,...,...,...,...,...,...
80425,گر,1,0,0,گر,گر,گر,NNP
80426,گروههای,1,0,0,گروههای,گروههای,گروههای,NNP
80427,گزارش,1,0,0,گزارش,گزارش,گزارش,NNP


In [20]:
def func(TOKEN, bag, tf_method, idf_method, count_method, count_item):
    TOKEN = TOKEN[~TOKEN.term_str.isna()]
    TOKEN['stem_porter'] = TOKEN.term_id.map(VOCAB.stem_porter)
    
    if count_item == 'term':
        print('term')
        BOW = TOKEN.groupby(bag+['term_id']).term_id.count().to_frame().rename(columns={'term_id':'n'})
        BOW['c'] = BOW.n.astype('bool').astype('int')
        
    else: #stem
        BOW = TOKEN.groupby(bag+['stem_porter']).stem_porter.count().to_frame().rename(columns={'stem_porter':'n'})
        BOW['c'] = BOW.n.astype('bool').astype('int')
    
    DTCM = BOW[count_method].unstack().fillna(0).astype('int')
    print('dtcm')
    if tf_method == 'sum':
        print('sum')
        TF = DTCM.T / DTCM.T.sum()

    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()

    elif tf_method == 'log':
        TF = np.log10(1 + DTCM.T)

    elif tf_method == 'raw':
        TF = DTCM.T

    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
        TF = 0.5 + (1 - 0.5) * TF[TF > 0] #switched tf_norm_k out as 0.5

    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
        
    print('done sum')
    TF = TF.T
    print('tf')

    DF = DTCM[DTCM > 0].count()
    print('idf')
    # compute IDF
    N = DTCM.shape[0]
    if idf_method == 'standard':
        print('standard')
        IDF = np.log10(N / DF)

    elif idf_method == 'max':
        IDF = np.log10(DF.max() / DF) 

    elif idf_method == 'smooth':
        IDF = np.log10((1 + N) / (1 + DF))

    print('multiply')
    TFIDF = TF * IDF
    return TFIDF, DF, IDF

In [21]:
OHCO = ['doc_id', 'sent_num', 'token_num']
#date - y m w, weekly cycles

In [22]:
BAG = OHCO[:1]
TFIDF, DF, IDF=func(TOKEN, BAG, 'sum', 'standard', 'n', 'term')
TFIDF.head()

/apps/software/standard/core/anaconda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


term
dtcm
sum
done sum
tf
idf
standard
multiply


term_id,0,1,2,3,4,5,6,7,8,9,...,80420,80421,80422,80423,80424,80425,80426,80427,80428,80429
doc_id,,,,,,,,,,,,,,,,,,,,,
48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
VOCAB['df'] = DF
VOCAB['idf'] = IDF

In [29]:
VOCAB

,term_id,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster,df,idf
0,0,0,28,1,0,0,0,0,24.0,2.609772
1,1,00,6,1,0,00,00,00,5.0,3.291013
2,2,000,4,1,0,000,000,000,3.0,3.512862
3,3,000000,1,1,0,000000,000000,000000,1.0,3.989983
4,4,00000001,1,1,0,00000001,00000001,00000001,1.0,3.989983
...,...,...,...,...,...,...,...,...,...,...
80425,80425,گر,1,0,0,گر,گر,گر,1.0,3.989983
80426,80426,گروههای,1,0,0,گروههای,گروههای,گروههای,1.0,3.989983
80427,80427,گزارش,1,0,0,گزارش,گزارش,گزارش,1.0,3.989983
80428,80428,ی,1,0,0,ی,ی,ی,1.0,3.989983


In [32]:
def tokenize(doc_df, OHCO=OHCO, remove_pos_tuple=False, ws=False):
    '''
    # Paragraphs to Sentences
    df = doc_df.sent_str\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(str(x))))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
        '''

    # Sentences to Tokens
    # Local function to pick tokenizer
    def word_tokenize(x):
        if ws:
            s = pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))
        else:
            s = pd.Series(nltk.pos_tag(nltk.word_tokenize(x))) # Discards stuff in between
        return s
            
    df = doc_df.sent_str\
        .apply(word_tokenize)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df